In [1]:
import torch

In [49]:
class Memory:

  def __init__(self, size):
    self.size = size
    self.stack = []

  def push(self, item):
    if len(self.stack) >= self.size:
      self.pop()
    self.stack.append(item)

  def pop(self):
    if self.stack:
      return self.stack.pop(0)
    else:
      return None

  def __repr__(self):
    return repr(self.stack)
  
  def copy(self):
    return self.stack.copy()
  
  def __iter__(self):
    return iter(self.stack[::-1])
  
  def full(self):
    return len(self.stack) == self.size

In [3]:
class Sweep():
  def __init__(self, processes, memory_dim):
    self.processes = processes
    self.n_processes = len(processes)
    self.memory_dim = memory_dim

  def make_dict(self):
    pass

In [4]:
class HawkesSweep(Sweep):

  def make_dict(self):
    dic = {}
    for i in range(self.n_processes):
      target = self.processes[i]
      dic[i] = {}
      for j in range(self.n_processes):
        cause = self.processes[j]
        dic[i][j] = self.sweep(target, cause)

    return dic

  def sweep(self, pa, pc):
    events = []
    pa_indices = []

    for i, ia in enumerate(pa):
      events.append((ia, 'a'))
      pa_indices.append(i)

    for ic in pc:
      events.append((ic, 'c'))

    lim = self.memory_dim

    events.sort()
    mem = Memory(lim)
    ret = []

    for t, e in events:
      if e == 'c':
        mem.push(t)

      if e == 'a':
        # Memory is not full yet
        if not mem.full():
          pp = [-1] * lim
        else:
          # Retrieve deltas from this time to the cause times
          pp = [t - tc for tc in mem]
        ret.append(pp)
    return torch.tensor(ret, dtype=torch.float)

In [5]:
hk = HawkesSweep([[1, 2, 3], [4, 5, 6]], 2)

hk.make_dict()

{0: {0: tensor([[-1., -1.],
          [-1., -1.],
          [ 1.,  2.]]),
  1: tensor([[-1., -1.],
          [-1., -1.],
          [-1., -1.]])},
 1: {0: tensor([[1., 2.],
          [2., 3.],
          [3., 4.]]),
  1: tensor([[-1., -1.],
          [-1., -1.],
          [ 1.,  2.]])}}

In [47]:
class WoldSweep(Sweep):
    def construct_wold_dict(self):
        dict = {}

        events = []
        for id, process in enumerate(self.processes):
            for t in process:
                events.append((t.item(), id))

        events.sort()

        deltas = {}
        last = {}
        cur = -1

        for t, id in events:
            dict[t] = {}
            deltas[id] = Memory(self.memory_dim)
            last[id] = [0, 0]

        for t, id in events:
            if t != cur:
                
                # updating
                cur = t
                for _id, _delta in deltas.items():
                    dict[cur][_id] = _delta.copy()
            
            last[id][1] = last[id][0]
            last[id][0] = t
            if last[id][1] != 0:
                deltas[id].push(last[id][0] - last[id][1])
            
        return dict
        
    # TODO: check idx_start semantics
    def make_dict(self):
        wold = self.construct_wold_dict()
        dic = {}
        for i in range(self.n_processes):
            target = self.processes[i]
            dic[i] = {}
            #for j in range(self.n_processes):
            #    cause = self.processes[j]
            #    dic[i][j] = self.sweep(target, cause)
            ret = {}
            for _t in target:
                t = _t.item()
                for j in range(self.n_processes):
                    if j not in ret:
                        ret[j] = []
                    #print(t, j, wold[t][j])
                    #return None
                    if len(wold[t][j]) < self.memory_dim:
                        ret[j].append([-1] * self.memory_dim)
                    else:
                        ret[j].append(wold[t][j])

            #return None
            for j in range(self.n_processes):
                dic[i][j] = torch.tensor(ret[j], dtype=torch.float)

        return dic


In [48]:
wd = WoldSweep(torch.tensor([[i for i in range(6)], [i for i in range(0, 6*4, 4)]]), 2)

wd.make_dict()

{0: {0: tensor([[-1., -1.],
          [-1., -1.],
          [-1., -1.],
          [-1., -1.],
          [ 1.,  1.],
          [ 1.,  1.]]),
  1: tensor([[-1., -1.],
          [-1., -1.],
          [-1., -1.],
          [-1., -1.],
          [-1., -1.],
          [-1., -1.]])},
 1: {0: tensor([[-1., -1.],
          [ 1.,  1.],
          [ 1.,  1.],
          [ 1.,  1.],
          [ 1.,  1.],
          [ 1.,  1.]]),
  1: tensor([[-1., -1.],
          [-1., -1.],
          [-1., -1.],
          [-1., -1.],
          [ 4.,  4.],
          [ 4.,  4.]])}}